In [1]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from datasets import load_dataset

# Load the MMLU dataset
dataset = load_dataset("cais/mmlu","all")

# Explore the dataset
print(dataset)


DatasetDict({
    test: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 1531
    })
    dev: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 285
    })
    auxiliary_train: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 99842
    })
})


In [20]:
import pandas as pd
test = pd.DataFrame(dataset['test'])
test['Name'] = 'MMLU_test'

In [21]:
val = pd.DataFrame(dataset['validation'])
val['Name'] = 'MMLU_val'

In [22]:
dev = pd.DataFrame(dataset['dev'])
dev['Name'] = 'MMLU_dev'

In [23]:
# 1: Combine the question and choices into a single column 'Question'
df = pd.concat([test,val,dev])
df['Question'] = df.apply(lambda row: f"{row['question']} The options are: " + 
                          ', '.join([f"{chr(65+i)}) {choice}" for i, choice in enumerate(row['choices'])]), axis=1)

# 2: Rename the columns
df.rename(columns={'subject': 'Category', 'answer': 'Correct Answer'}, inplace=True)

# 3: Change the value of 'Correct Answer' to A, B, C, D
df['Correct Answer'] = df['Correct Answer'].apply(lambda x: chr(65 + x))

# Selecting the required columns
df_final = df[['Name', 'Category', 'Question', 'Correct Answer']]

df_final

,Name,Category,Question,Correct Answer
0,MMLU_test,abstract_algebra,Find the degree for the given field extension ...,B
1,MMLU_test,abstract_algebra,"Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the i...",C
2,MMLU_test,abstract_algebra,Find all zeros in the indicated finite field o...,D
3,MMLU_test,abstract_algebra,Statement 1 | A factor group of a non-Abelian ...,B
4,MMLU_test,abstract_algebra,Find the product of the given polynomials in t...,B
...,...,...,...,...
280,MMLU_dev,world_religions,What is the sign of the covenant for Jewish m...,B
281,MMLU_dev,world_religions,What is the Second Gem in Buddhism? The option...,A
282,MMLU_dev,world_religions,"In which dynasty was the ""Mandate of Heaven"" ...",B
283,MMLU_dev,world_religions,Which Japanese government promoted a kind of ...,D


In [24]:
df_final.to_csv('../preprocessed/MMLU.csv',index = False)